In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchinfo import summary
import pandas as pd
import numpy as np

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, path):
        df = pd.read_csv(path)
        self.X = torch.tensor(df.loc[: ,df.columns != "label"].values.astype('float32'))
        m = self.X.mean()
        v = torch.sqrt(self.X.var())
        self.X = (self.X - m) / v
        self.X = self.X.view(self.X.size()[0], 1, 28, 28)
        if "label" in df.columns:
            self.y = torch.tensor(df["label"].values)
        else:
            self.y = torch.zeros(self.X.size()[0])

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        return (X, y)

In [4]:
dataset_train = MyDataset("/kaggle/input/digit-recognizer/train.csv")
dataset_test = MyDataset("/kaggle/input/digit-recognizer/test.csv")

In [5]:
train_size = round(len(dataset_train) * 0.7)
test_size = len(dataset_train) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset_train, [train_size, test_size])

train_dl = DataLoader(train_dataset, batch_size=64, shuffle=True) # shuffle = True !!!
test_dl = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [6]:
class ModelConvMNIST(nn.Module):
  def __init__(self, input_chanel = 1, output_lables = 10):
    super(ModelConvMNIST, self).__init__()
    self.input_chanel = input_chanel
    self.output_lables = output_lables
    self.relu = nn.LeakyReLU(0.2)
    self.dropout = nn.Dropout(0.3)
    self.flatten = nn.Flatten()
    self.max_pool = nn.MaxPool2d(2)
    
    self.conv1 = nn.Conv2d(1, 64, 3, 1, 1)
    self.batch_norm1 = nn.BatchNorm2d(64)
    
    self.conv2 = nn.Conv2d(64, 128, 3)
    self.batch_norm2 = nn.BatchNorm2d(128)
    
    self.conv3 = nn.Conv2d(128, 128, 3, 1, 1)
    self.batch_norm3 = nn.BatchNorm2d(128)
    
    self.conv4 = nn.Conv2d(128, 256, 3)
    self.batch_norm4 = nn.BatchNorm2d(256)
    
    self.conv5 = nn.Conv2d(256, 256, 3)
    self.batch_norm5 = nn.BatchNorm2d(256)
    
    self.conv6 = nn.Conv2d(256, 512, 3)
    self.batch_norm6 = nn.BatchNorm2d(512)

    self.linear1 = nn.Linear(512, 1000)
    self.linear2 = nn.Linear(1000, 10)
    self.sigmoid = nn.Sigmoid()

  def forward(self, input_image):
    output = self.batch_norm1(self.relu(self.conv1(input_image)))
    output = self.dropout(self.batch_norm2(self.max_pool(self.relu(self.conv2(output)))))
    output = self.batch_norm3(self.relu(self.conv3(output)))
    output = self.batch_norm4(self.max_pool(self.relu(self.conv4(output))))
    output = self.dropout(self.batch_norm5(self.relu(self.conv5(output))))
    output = self.batch_norm6(self.relu(self.conv6(output)))
    output = self.relu(self.linear1(self.flatten(output)))
    output = self.linear2(output)
    return self.sigmoid(output)

In [7]:
model = ModelConvMNIST().to(device)
BATCH_SIZE = 64
summary(model, input_size=(BATCH_SIZE, 1, 28, 28))    

Layer (type:depth-idx)                   Output Shape              Param #
ModelConvMNIST                           [64, 10]                  --
├─Conv2d: 1-1                            [64, 64, 28, 28]          640
├─LeakyReLU: 1-2                         [64, 64, 28, 28]          --
├─BatchNorm2d: 1-3                       [64, 64, 28, 28]          128
├─Conv2d: 1-4                            [64, 128, 26, 26]         73,856
├─LeakyReLU: 1-5                         [64, 128, 26, 26]         --
├─MaxPool2d: 1-6                         [64, 128, 13, 13]         --
├─BatchNorm2d: 1-7                       [64, 128, 13, 13]         256
├─Dropout: 1-8                           [64, 128, 13, 13]         --
├─Conv2d: 1-9                            [64, 128, 13, 13]         147,584
├─LeakyReLU: 1-10                        [64, 128, 13, 13]         --
├─BatchNorm2d: 1-11                      [64, 128, 13, 13]         256
├─Conv2d: 1-12                           [64, 256, 11, 11]         295,1

In [8]:
class ModelMNIST(nn.Module):
  def __init__(self, input_chanel = 1, output_lables = 10):
    super(ModelMNIST, self).__init__()
    self.input_chanel = input_chanel
    self.output_lables = output_lables
    self.relu = nn.LeakyReLU(0.2)
    self.dropout = nn.Dropout(0.3)
    self.linear1 = nn.Linear(784, 1024)
    self.batch_norm1 = nn.BatchNorm1d(1024)
    self.linear2 = nn.Linear(1024, 2048)
    self.batch_norm2 = nn.BatchNorm1d(2048)
    self.linear3 = nn.Linear(2048, 512)
    self.batch_norm3 = nn.BatchNorm1d(512)
    self.linear4 = nn.Linear(512, 10)
    self.sigmoid = nn.Sigmoid()

  def forward(self, input_image):
    input_image = input_image.view(-1, 784)
    output = self.batch_norm1(self.relu(self.linear1(input_image)))
    output = self.dropout(self.batch_norm2(self.relu(self.linear2(output))))
    output = self.batch_norm3(self.relu(self.linear3(output)))
    output = self.linear4(output)
    return self.sigmoid(output)

In [9]:
model = ModelMNIST().to(device)

In [10]:
epochs = 20
lr = 10e-6

In [11]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [12]:
for epoch in range(epochs):
    test_loss = 0
    train_loss = 0
    
    train_correct = 0
    val_correct = 0
    
    model.eval()
    with torch.no_grad():
        for i, (X, y) in enumerate(train_dl):
            y_pred = model(X.to(device))
            l = loss(y_pred, y.to(device))
            train_loss += l.item() * len(y)
            train_correct += (y_pred.argmax(dim=1) == y.to(device)).float().sum()
            
        for i, (X, y) in enumerate(test_dl):
            y_pred = model(X.to(device))
            l = loss(y_pred, y.to(device))
            test_loss += l.item() * len(y)
            val_correct += (y_pred.argmax(dim=1) == y.to(device)).float().sum()
            
        avg_train_loss = train_loss / len(train_dataset)
        train_acc = train_correct / len(train_dataset)

        avg_val_loss = test_loss / len(test_dataset)
        val_acc = val_correct / len(test_dataset)

        print(f'epoch: {epoch} - train_loss: {avg_train_loss:.5f} - train_acc: {train_acc:.5f} - val_loss: {avg_val_loss:.5f} - val_acc: {val_acc:.5f}')
            
        model.train()


    for i, (X, y) in enumerate(train_dl):
        y_pred = model(X.to(device))
        l = loss(y_pred, y.to(device))
        l.backward()

        optimizer.step()
        optimizer.zero_grad()

epoch: 0 - train_loss: 2.30330 - train_acc: 0.15391 - val_loss: 2.30332 - val_acc: 0.15794
epoch: 1 - train_loss: 1.80118 - train_acc: 0.88197 - val_loss: 1.80259 - val_acc: 0.88095
epoch: 2 - train_loss: 1.75614 - train_acc: 0.88656 - val_loss: 1.75848 - val_acc: 0.88619
epoch: 3 - train_loss: 1.72559 - train_acc: 0.88935 - val_loss: 1.72863 - val_acc: 0.88706
epoch: 4 - train_loss: 1.69899 - train_acc: 0.89459 - val_loss: 1.70274 - val_acc: 0.89230
epoch: 5 - train_loss: 1.67669 - train_acc: 0.89531 - val_loss: 1.68124 - val_acc: 0.89246
epoch: 6 - train_loss: 1.65954 - train_acc: 0.90054 - val_loss: 1.66462 - val_acc: 0.89603
epoch: 7 - train_loss: 1.64296 - train_acc: 0.90170 - val_loss: 1.64885 - val_acc: 0.89627
epoch: 8 - train_loss: 1.62669 - train_acc: 0.90102 - val_loss: 1.63307 - val_acc: 0.89611
epoch: 9 - train_loss: 1.61334 - train_acc: 0.90415 - val_loss: 1.62027 - val_acc: 0.89984
epoch: 10 - train_loss: 1.60217 - train_acc: 0.90592 - val_loss: 1.60969 - val_acc: 0.8982

In [13]:
result_dl = DataLoader(dataset_test, batch_size=16, shuffle=False)

In [14]:
Y = torch.tensor([])
model.eval()
for i, (X, _) in enumerate(result_dl):
    y_pred = model(X.to(device)).cpu().detach()
    Y = torch.cat((Y, y_pred), dim = 0)

In [15]:
Y = Y.argmax(dim=1).numpy()

In [16]:
df = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")

In [17]:
df["Label"] = Y

In [18]:
df

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,4
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,4


In [19]:
df.to_csv("result_MNIST_5.csv", index=False)